In [ ]:
%run **/other_files/task2.ipynb

# Task 3: Baseline Model Comparison (Logistic Regression)

In this notebook, you will train a Logistic Regression model to predict the `prior_hiring_decision` target variable. You will also begin your fairness analysis by establishing a baseline for accuracy and fairness metrics.

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load Data
train_df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv('../data/val.csv')
test_df = pd.read_csv('../data/test.csv')

columns_to_drop = ["Age", "Sex", "Race", "Place_Of_Birth", "Hours_Per_Week", "Marital_Status", "Relationship"]

for df in [train_df, val_df, test_df]:
    df.drop(columns=columns_to_drop, inplace=True)
print("Training Shape:", train_df.shape)

Training Shape: (30000, 6)


## 1. Preprocessing

In [ ]:
# Define target and features
target = 'prior_hiring_decision'
features = [col for col in train_df.columns if col != target]

X_train = train_df[features]
y_train = train_df[target]
X_val = val_df[features]
y_val = val_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Define column types
categorical_cols = ['Workclass', 'Sex', 'Race', 'Marital_Status', 'Education', 'Occupation', 'Relationship', 'Place_Of_Birth']
numerical_cols = [col for col in features if col not in categorical_cols]

# Preprocessing: scale numerical, one-hot encode categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Preprocess data
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# Hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['saga']
}


## 2. Preprocessing Fairness

In [ ]:
def compute_intersectional_weights(df, protected_cols, label_col):
    """
    Compute weights for intersectional groups (combinations of all protected characteristics).
    W_{group, label} = (Count(Group) Ã— Count(Label)) / (Total Ã— Count(Group âˆ© Label))
    """
    total = len(df)
    labels = df[label_col].unique()
    
    # Create intersectional group column
    df = df.copy()
    df['_group'] = df[protected_cols].astype(str).agg('_'.join, axis=1)
    
    groups = df['_group'].unique()
    
    weights = {}
    for group in groups:
        group_count = len(df[df['_group'] == group])
        for label in labels:
            label_count = len(df[df[label_col] == label])
            intersection_count = len(df[(df['_group'] == group) & (df[label_col] == label)])
            
            if intersection_count > 0:
                weight = (group_count * label_count) / (total * intersection_count)
            else:
                weight = 1.0
            
            weights[(group, int(label))] = float(weight)
    
    return weights, df['_group']

def compute_sample_weights(df, protected_cols, label_col):
    """
    Compute per-sample weights based on intersectional group-label combinations.
    """
    group_weights, group_col = compute_intersectional_weights(df, protected_cols, label_col)
    
    sample_weights = []
    for idx, row in df.iterrows():
        group_key = '_'.join([str(row[col]) for col in protected_cols])
        label = int(row[label_col])
        weight = group_weights.get((group_key, label), 1.0)
        sample_weights.append(weight)
    
    return np.array(sample_weights), group_weights

# Compute intersectional weights
protected_cols = ['Sex', 'Race']
target = 'prior_hiring_decision'

sample_weights, group_weights = compute_sample_weights(train_df, protected_cols, target)

print("Intersectional Group Weights:")
for (group, label), weight in sorted(group_weights.items()):
    print(f"  Group={group}, Label={label}: {weight:.4f}")

print(f"\nSample weights computed for {len(train_df)} samples")
print(f"Weight range: [{sample_weights.min():.4f}, {sample_weights.max():.4f}]")
print(f"Unique weights: {len(set(sample_weights))}")

## 3. Training

In [ ]:

best_score = 0
best_params = {}

for C in param_grid['C']:
    for penalty in param_grid['penalty']:
        clf = LogisticRegression(
            C=C,
            penalty=penalty,
            solver='saga',
            max_iter=1000,
            random_state=42
        )
        clf.fit(X_train_processed, y_train)
        score = clf.score(X_val_processed, y_val)
        if score > best_score:
            best_score = score
            best_params = {'C': C, 'penalty': penalty}

print(f"Best params: {best_params}")
print(f"Best validation accuracy: {best_score:.4f}")

# Train final model
best_clf = LogisticRegression(**best_params, solver='saga', max_iter=1000, random_state=42)
best_clf.fit(X_train_processed, y_train)
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', best_clf)
])

## 4. Evaluation

In [4]:
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {acc:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

Validation Accuracy: 0.7316
Confusion Matrix:
[[3924 1275]
 [1409 3392]]


/Users/matthewwicker/Desktop/Development/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/matthewwicker/Desktop/Development/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/matthewwicker/Desktop/Development/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


## 5. Fairness Metric Implementation

In [ ]:
protected_characteristics = ['Sex', 'Race', 'Age']

# Add predictions to validation data for fairness evaluation
val_with_preds = val_df.copy()
val_with_preds['prediction'] = best_clf.predict(X_val_processed)

def compute_fairness_metrics(data, protected_characteristics, target='prediction'):
    results = {
        'demographic_parity': {},
        'equalized_opportunity': {},
        'equalized_odds': {}
    }
    
    for char in protected_characteristics:
        groups = data[char].unique()
        
        # Demographic Parity
        dp_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            dp_info[int(group)] = {
                'positive_rate': float(group_data[target].mean()),
                'count': int(len(group_data))
            }
        dp_info['ratio'] = float(demographic_parity(char, target, data))
        results['demographic_parity'][char] = dp_info
        
        # Equalized Opportunity
        eo_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            positive_cases = group_data[group_data[target] == 1]
            eo_info[int(group)] = {
                'tpr': float(positive_cases[target].mean()) if len(positive_cases) > 0 else None,
                'positive_count': int(len(positive_cases))
            }
        eo_info['ratio'] = float(equalized_opportunity(char, target, data))
        results['equalized_opportunity'][char] = eo_info
        
        # Equalized Odds
        eod_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            positive_cases = group_data[group_data[target] == 1]
            negative_cases = group_data[group_data[target] == 0]
            eod_info[int(group)] = {
                'tpr': float(positive_cases[target].mean()) if len(positive_cases) > 0 else None,
                'fpr': float(negative_cases[target].mean()) if len(negative_cases) > 0 else None,
            }
        eod_info['ratio'] = float(equalized_odds(char, target, data))
        results['equalized_odds'][char] = eod_info
    
    return results

fairness_results = compute_fairness_metrics(val_with_preds, protected_characteristics, 'prediction')
print("Fairness Metrics (Logistic Regression):")
print("\nDemographic Parity:", fairness_results['demographic_parity'])
print("\nEqualized Opportunity:", fairness_results['equalized_opportunity'])
print("\nEqualized Odds:", fairness_results['equalized_odds'])

## 6. ROC Curve Analysis

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_proba = best_clf.predict_proba(X_val_processed)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_proba)
auc_score = roc_auc_score(y_val, y_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

print(f"AUC Score: {auc_score:.4f}")

## 7. Adjusted Equalized Odds

In [ ]:
def adjusted_equalized_odds(data, protected_col, y_true_col, y_proba, thresholds):
    """
    Calculate equalized odds using group-specific classification thresholds.
    
    Args:
        data: DataFrame with protected characteristic and true labels
        protected_col: Name of protected characteristic column
        y_true_col: Name of true label column
        y_proba: Array of predicted probabilities
        thresholds: Dict mapping group values to classification thresholds
    """
    data = data.copy()
    data['y_proba'] = y_proba
    
    data['adjusted_pred'] = data.apply(
        lambda row: 1 if row['y_proba'] >= thresholds.get(row[protected_col], 0.5) else 0,
        axis=1
    )
    
    groups = data[protected_col].unique()
    tpr_rates = []
    fpr_rates = []
    results = {}
    
    for group in groups:
        group_data = data[data[protected_col] == group]
        positive_cases = group_data[group_data[y_true_col] == 1]
        negative_cases = group_data[group_data[y_true_col] == 0]
        
        tpr = positive_cases['adjusted_pred'].mean() if len(positive_cases) > 0 else 0
        fpr = negative_cases['adjusted_pred'].mean() if len(negative_cases) > 0 else 0
        
        tpr_rates.append(tpr)
        fpr_rates.append(fpr)
        
        results[int(group)] = {
            'tpr': float(tpr),
            'fpr': float(fpr),
            'threshold': float(thresholds.get(group, 0.5)),
            'count': int(len(group_data))
        }
    
    max_tpr, min_tpr = max(tpr_rates), min(tpr_rates)
    max_fpr, min_fpr = max(fpr_rates), min(fpr_rates)
    
    tpr_ratio = max_tpr / min_tpr if min_tpr > 0 else float('inf')
    fpr_ratio = max_fpr / min_fpr if min_fpr > 0 else float('inf')
    
    results['tpr_ratio'] = float(tpr_ratio)
    results['fpr_ratio'] = float(fpr_ratio)
    results['equalized_odds_ratio'] = float(max(tpr_ratio, fpr_ratio))
    
    return results